In [180]:
import numpy as np
import torch

def coarsening_node_feat(node_feat, c_source_node, c_target_node): # Idea...
  node_feat = node_feat.tolist()
  source_node_feat = node_feat[c_source_node]
  target_node_feat = node_feat[c_target_node]
  
  new_node_feat = source_node_feat + target_node_feat
  mark_del_node_feat = [-1]*len(new_node_feat)

  node_feat[c_source_node] = mark_del_node_feat
  node_feat[c_target_node] = mark_del_node_feat
  node_feat.append(new_node_feat)
  

  node_feat = [item + [0] * (len(new_node_feat) - len(item)) for item in node_feat]
  node_feat = np.array(node_feat)

  return node_feat

def coarsening_edge_attr(edge_index, edge_attr, c_source_node, c_target_node): # Idea...

  c_edge_indices = np.where((edge_index[:, 0] == c_source_node) | (edge_index[:, 0] == c_target_node))[0]
  target_node = edge_index[c_edge_indices, 1]
  target_node_indices = [np.where(target_node == node)[0] for node in np.unique(target_node)]
  
  for node, indices in zip(np.unique(target_node), target_node_indices):
    if len(c_edge_indices[indices]) > 1 :
        edge_attr[c_edge_indices[indices]] = np.sum(edge_attr[c_edge_indices[indices]], axis=0)
         
  c_edge_indices = np.where((edge_index[:, 1] == c_source_node) | (edge_index[:, 1] == c_target_node))[0]
  source_node = edge_index[c_edge_indices, 0]
  source_node_indices = [np.where(source_node == node)[0] for node in np.unique(source_node)]

  for node, indices in zip(np.unique(source_node), source_node_indices):
    if len(c_edge_indices[indices]) > 1 :
        edge_attr[c_edge_indices[indices]] = np.sum(edge_attr[c_edge_indices[indices]], axis=0)

  return edge_attr

In [185]:
node_feat = np.array([[0,0,0,0],[1,1,1,1],[2,2,2,2],[3,3,3,3],[4,4,4,4],[5,5,5,5]])
edge_index = np.array([[0,0,1,2,3,3,3,3,4,4,4,5,5,5],[3,5,4,3,0,2,4,5,1,3,5,0,3,4]])
edge_attr = np.array([[1,0,2],[1,0,1],[3,0,1],[2,0,5],[1,0,2],[2,0,5],[6,0,1],[0,0,2],[3,0,1],[6,0,1],[4,0,5],[1,0,1],[0,0,2],[4,0,5]])

r = 0.5

print("Original edge_index \n",edge_index)

num_edge = len(edge_index.T)
edge_index = edge_index.T
new_node = np.max(edge_index)
coarsened_edge = np.sort(np.random.choice(range(0, num_edge), int(num_edge * r/2), replace = False))
mark_del_edge_index = np.full(edge_index.shape[1], -1)
mark_del_edge_feat = np.full(edge_attr.shape[1], -1) 

Original edge_index 
 [[0 0 1 2 3 3 3 3 4 4 4 5 5 5]
 [3 5 4 3 0 2 4 5 1 3 5 0 3 4]]


In [186]:
# Check whether coarsened_edge includes some two edges that are same in undirected graph
# coarsened_edge = np.array([4, 6, 11])

for idx_c_edge in range(len(coarsened_edge)):
   
  dup_edge = np.array([edge_index[coarsened_edge[idx_c_edge]][1], edge_index[coarsened_edge[idx_c_edge]][0]])
  
  for j in range(len(coarsened_edge)):
    if j != idx_c_edge:
             
       if (edge_index[coarsened_edge[j]] == dup_edge).all():
          coarsened_edge = np.delete(coarsened_edge, j)
          break

  if idx_c_edge+1 == len(coarsened_edge):
    break

print("Which edges will be deleted \n",coarsened_edge)

Which edges will be deleted 
 [ 2  9 12]


In [187]:
# Change edge_index & node_feat

for idx_c_edge in coarsened_edge:
  
  new_node += 1

  if edge_index[idx_c_edge, 0] > edge_index[idx_c_edge, 1]:
     c_source_node = edge_index[idx_c_edge, 1]
     c_target_node = edge_index[idx_c_edge, 0]
  else:
     c_source_node = edge_index[idx_c_edge, 0]
     c_target_node = edge_index[idx_c_edge, 1]
  
  if c_source_node == -1 or c_target_node == -1:
    continue

  node_feat = coarsening_node_feat(node_feat, c_source_node, c_target_node)

  # Updating edge features for coarsened edges
  edge_attr = coarsening_edge_attr(edge_index, edge_attr, c_source_node, c_target_node)
  
  # Replacing coarsened node index to new node index
  mask_source = np.logical_or(edge_index[:, 0] == c_source_node, edge_index[:, 0] == c_target_node)
  mask_target = np.logical_or(edge_index[:, 1] == c_source_node, edge_index[:, 1] == c_target_node)
  edge_index[mask_source, 0] = new_node
  edge_index[mask_target, 1] = new_node

  # Marking deleted edge indices and features by [-1, -1] and [-1, -1, ... , -1]
  mask_self_loop = edge_index[:, 0] == edge_index[:, 1]
  edge_index[mask_self_loop] = mark_del_edge_index
  edge_attr[mask_self_loop] = mark_del_edge_feat

In [188]:
# Node feature part

rows_to_delete = np.where(node_feat[:, 0] == -1) 
node_feat = np.delete(node_feat, rows_to_delete, axis=0) # Delete [-1, ... , -1]

# Edge feature part

edge_dict = {}
for i, edge in enumerate(edge_index):
    key = tuple(edge)
    if key in edge_dict:
        if np.all(edge_attr[edge_dict[key]] == mark_del_edge_feat):
            continue
        else:
            edge_attr[edge_dict[key]] += edge_attr[i]
    else:
        edge_dict[key] = i
unique_edge_attr = edge_attr[list(edge_dict.values())] # Aggregate edge features
edge_attr = unique_edge_attr[~np.all(unique_edge_attr == mark_del_edge_feat, axis = 1)] # Delete [-1, ... -1]

rows_to_delete = np.where(np.all(edge_index == mark_del_edge_index, axis=1))
edge_index = np.delete(edge_index, rows_to_delete, axis=0) # Delete [-1, -1]
edge_index, unique_indices = np.unique(edge_index, return_index=True, axis = 0) # Delete duplicated node pairs
edge_index = edge_index[np.argsort(unique_indices)].T

print("node_feat \n",node_feat)
print("edge_index \n",edge_index)
print("edge_attr \n",edge_attr)

node_feat 
 [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [5 5 5 5 0 0 0 0 0 0 0 0 0 0 0 0 3 3 3 3 0 0 0 0 1 1 1 1 4 4 4 4]]
edge_index 
 [[0 2 8 8]
 [8 8 0 2]]
edge_attr 
 [[4 0 6]
 [2 0 5]
 [4 0 6]
 [2 0 5]]


In [189]:
# Reindex edge_index

flattened_edge_index = edge_index.flatten()
unique_elements = np.unique(flattened_edge_index)
element_mapping = {element: new_value for new_value, element in enumerate(unique_elements)}
edge_index = np.vectorize(element_mapping.get)(edge_index)

print("Final node_feat \n",node_feat)
print("Final edge_index \n",edge_index)
print("Final edge_attr \n",edge_attr)

Final node_feat 
 [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [5 5 5 5 0 0 0 0 0 0 0 0 0 0 0 0 3 3 3 3 0 0 0 0 1 1 1 1 4 4 4 4]]
Final edge_index 
 [[0 1 2 2]
 [2 2 0 1]]
Final edge_attr 
 [[4 0 6]
 [2 0 5]
 [4 0 6]
 [2 0 5]]


In [23]:
import numpy as np

class Graph:
    def __init__(self, num_nodes, edge_index):
        self.num_nodes = num_nodes
        self.edge_index = edge_index

def coarsen_graph(graph):
    # Simplified coarsening: Merge pairs of nodes into super-nodes
    num_coarsened_nodes = graph.num_nodes // 2
    coarsened_edge_index = [[-1, -1] for _ in range(num_coarsened_nodes * 2)]
    
    i = 0
    while i < len(graph.edge_index[0]):
        source1, target1 = graph.edge_index[0][i], graph.edge_index[1][i]
        
        if source1 != -1:
            source2, target2 = graph.edge_index[0][i+1], graph.edge_index[1][i+1]
            if source2 != -1:
                if coarsened_edge_index[source1][0] == -1:
                    coarsened_edge_index[source1][0] = target1
                else:
                    coarsened_edge_index[source1][1] = target1
                
                if coarsened_edge_index[target1][0] == -1:
                    coarsened_edge_index[target1][0] = source1
                else:
                    coarsened_edge_index[target1][1] = source1
                
                i += 2
            else:
                coarsened_edge_index[source1][0] = target1
                coarsened_edge_index[target1][0] = source1
                i += 1
        else:
            i += 1
    
    # Remove duplicate connections
    unique_coarsened_edge_index = []
    for i in range(num_coarsened_nodes * 2):
        if coarsened_edge_index[i][0] != -1:
            unique_coarsened_edge_index.append(coarsened_edge_index[i])
    
    return Graph(num_coarsened_nodes, unique_coarsened_edge_index)

def uncoarsen_graph(coarsened_graph, fine_graph):
    # Upscale the coarsened graph to the fine graph
    num_fine_nodes = fine_graph.num_nodes
    fine_edge_index = [[-1, -1] for _ in range(num_fine_nodes * 2)]
    
    for i in range(len(coarsened_graph.edge_index)):
        source, target = coarsened_graph.edge_index[i]
        
        if source != -1:
            source *= 2
            target *= 2
            
            fine_edge_index[source][0] = target
            fine_edge_index[target][0] = source
    
    return Graph(num_fine_nodes, fine_edge_index)

# Example edge_index representing pairs of edges
edge_index = np.array([
    [0, 0, 1, 2, 3, 3, 3, 3, 4, 4, 4, 5, 5, 5],
    [3, 5, 4, 3, 0, 2, 4, 5, 1, 3, 5, 0, 3, 4]
])

original_graph = Graph(6, edge_index)

# Coarsen the original graph
coarsened_graph = coarsen_graph(original_graph)

# Uncoarsen the coarsened graph back to the original graph
restored_graph = uncoarsen_graph(coarsened_graph, original_graph)

# Print the edge_index of the original and restored graphs
print("Original Edge Index:")
print(original_graph.edge_index)

print("\nCoarsened Edge Index:")
print(coarsened_graph.edge_index)

print("\nRestored Edge Index:")
print(restored_graph.edge_index)

Original Edge Index:
[[0 0 1 2 3 3 3 3 4 4 4 5 5 5]
 [3 5 4 3 0 2 4 5 1 3 5 0 3 4]]

Coarsened Edge Index:
[[3, 3], [4, 4], [0, 5], [1, 5], [4, 3]]

Restored Edge Index:
[[10, -1], [-1, -1], [10, -1], [-1, -1], [-1, -1], [-1, -1], [8, -1], [-1, -1], [6, -1], [-1, -1], [2, -1], [-1, -1]]
